In [1]:
pip install jax jaxlib flax transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
  Using cached msgpack-1.0.8-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.8 MB/s eta 0:00:00
  Using cached safetensors-0.4.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (3.8 kB)
  Using cached tokenizers-0.19.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.7 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.9 MB/s eta 0:00:00
  Using cached chex-0.1.86-py3-none-any.whl.metadata (17 kB)
  Using cached toolz-0.12.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached importlib_resources-6.4.0-py3-none-any.whl.metadata (3.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 52.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.3/731.3 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 

In [ ]:
import jax
import jax.numpy as jnp
from flax import linen as nn
from transformers import BertTokenizer, FlaxBertModel

In [34]:
class DualEncoder(nn.Module):
    model_name: str

    def setup(self):
        self.encoder = FlaxBertModel.from_pretrained(self.model_name)

    @nn.compact
    def __call__(self, input_ids, attention_mask, token_type_ids):
        outputs = self.encoder(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        return outputs.last_hidden_state[:, 0]  # Use [CLS] token

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [35]:
sentences = ["This is a test sentence.", "Another sentence for testing."]
tokenized_inputs = tokenizer(sentences, padding='max_length', max_length=64, return_tensors='jax')
input_ids = tokenized_inputs['input_ids']
attention_mask = tokenized_inputs['attention_mask']
token_type_ids = tokenized_inputs['token_type_ids']

In [36]:
# Create an instance of the model
model = DualEncoder(model_name='bert-base-multilingual-cased')

# Initialize the model parameters
rng = jax.random.PRNGKey(0)
variables = model.init(rng, input_ids, attention_mask, token_type_ids)

# Bind the model to its variables
bound_model = model.bind(variables)

flax_model.msgpack:   0%|          | 0.00/712M [00:00<?, ?B/s]

Some weights of FlaxBertModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: {('pooler', 'dense', 'bias'), ('pooler', 'dense', 'kernel')}
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
model()

CallCompactUnboundModuleError: Can't call compact methods on unbound modules (https://flax.readthedocs.io/en/latest/api_reference/flax.errors.html#flax.errors.CallCompactUnboundModuleError)

In [13]:
e = model.encoder

In [14]:
type(e)

NoneType

In [11]:
e